# LIBRERIE

In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

# DATASET

In [38]:
dataset = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/DATASET/CosciaRios2012_DataBase.csv') #lettura del dataset
dataset.drop('Code', axis=1, inplace=True) #rimuoviamo la collona dove sono presenti i codici "provinciali"
dataset = dataset.groupby(['State','Year']).sum() #sommiamo le nostre istanze in base a Stato e Anno
dataset.to_csv('Mexican_Drug_Cartels_dta.csv') #salviamo il dataset

In [39]:
dataset = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/Mexican_Drug_Cartels_dta.csv')
#dividiamo il nostro datastet in due decadi 
dta_9000 = dataset[dataset.Year < 2000] #1990-1999
dta_0010 = dataset[dataset.Year >= 2000 ] #2000-2010

# FUNZIONI

In [62]:
def Edges_Generetor(dataset, name_txt, Qcartelli, stateL=False, yearL=False, yearXstate=False, cartelL=False):
    #permette la creazione di edges partendo dal dataset di partenza

    #esegue un check per vedere crearli su tutte le famiglie o su una in particolare
    if Qcartelli == 'ALL':
        cartelli = ['Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia', 'Golfo', 
                    'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas', 'Otros']
    else:
        cartelli = []
        cartelli.append(Qcartelli)

    res_list = []
    
    for state in dataset.State.unique(): #creiuamo un subset per ogni stato in maniera ricorsiva
        sub1 = dataset[dataset.State == state]

        for year in sub1.Year: #creiuamo un subset per ogni anno in maniera ricorsiva
            sub2 = sub1[sub1.Year == year]
            
            for single in cartelli:
                value = sub2[single].iloc[0] #troviamo il numero di volte che il cartello è apparso
                list = []

                if value > 0 : #eseguiamo un controllo per sapere se creare o no il collegamento
                    #in base alle impostazioni decise prendiamo PARTENZA(A), ARRIVO(B) e WEIGHT(W)
                    if yearXstate == True:
                        year_ = str(year)
                        _state = str(state)
                        year_state = str(year_ + '_' + _state)

                    if stateL == True:
                        list.append(state)
                    
                    if yearL == True:
                        list.append(year)
                    elif yearXstate == True:
                        list.append(year_state)
                    
                    if Qcartelli == 'ALL':
                        if cartelL == True:
                            list.append(single)
                    else:
                        if cartelL == True:
                            list.append(Qcartelli)
                    
                    list.append(value)
                    res_list.append(list)

    #creiamo il file.txt con il nome deciso nelle impostazioni                    
    name_file = str(name_txt + ".txt")
    file = open(name_file, "w+")

    file.write('A'+',')
    file.write('B'+',')
    file.write('Weight'+'\n')

    for list in res_list:
        time = 0
        for element in list:
            time = time + 1
            e = str(element)
            file.write(e)
            if time < 3:
                file.write(',')
            else:
                file.write('\n')

    file.close()

    return(file)

#----------------------------------------------------------------------------------------#

def Edges_Generetor_for_Cartel(name_cartel, cartel_sigla):

    name_txt_9000_yXc = 'year_'+str(name_cartel)+'_'+str(cartel_sigla)+'9000'
    name_txt_9000_sXy = 'state_year_'+str(name_cartel)+'_'+str(cartel_sigla)+'9000'

    Edges_Generetor(dta_9000,name_txt_9000_yXc, name_cartel, yearL=True,cartelL=True)
    Edges_Generetor(dta_9000,name_txt_9000_sXy, name_cartel, stateL=True, yearL=True)

    name_txt_9000_yXc = name_txt_9000_yXc + '.txt'
    name_txt_9000_sXy = name_txt_9000_sXy + '.txt'

    percorso_cartella = '/Users/emanuele/Desktop/SNA/PROGETTO/'
    percorso_1 = str(percorso_cartella) + str(name_txt_9000_sXy)
    percorso_2 = str(percorso_cartella) + str(name_txt_9000_yXc)

    df_1 = pd.read_csv(str(percorso_1))
    df_2 = pd.read_csv(str(percorso_2))
    df = pd.concat([df_1,df_2],ignore_index=True, sort=False)

    return(df)

 

# NETWORK VISUALIZATION

## GENERAL

In [36]:
Edges_Generetor(dta_9000,'G_yearXstate_cartel_9000','ALL',yearXstate=True,cartelL=True) #G_yearXstate_cartel_9000
Edges_Generetor(dta_9000,'G_state_yearXstate_9000','ALL',stateL=True, yearXstate=True) #G_state_yearXstate_9000
df_1 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/G_state_yearXstate_9000.txt')
df_2 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/G_yearXstate_cartel_9000.txt')
df_9000 = pd.concat([df_1,df_2],ignore_index=True, sort=False)


Edges_Generetor(dta_0010,'G_yearXstate_cartel_0010','ALL',yearXstate=True,cartelL=True) #G_yearXstate_cartel_0010
Edges_Generetor(dta_0010,'G_state_yearXstate_0010','ALL',stateL=True, yearXstate=True) #G_state_yearXstate_0010
df_1 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/G_state_yearXstate_0010.txt')
df_2 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/G_yearXstate_cartel_0010.txt')
df_0010 = pd.concat([df_1,df_2],ignore_index=True, sort=False)

## PER FAMIGLIA

Bertran Leyva --> 9000 no 
Bertran Leyva Family --> 9000 no


In [41]:
dta_0010.columns

Index(['State', 'Year', 'Beltran_Leyva', 'Beltran_Leyva_Family', 'Familia',
       'Golfo', 'Juarez', 'Sinaloa', 'Sinaloa_Family', 'Tijuana', 'Zetas',
       'Otros'],
      dtype='object')

In [43]:
Edges_Generetor(dta_0010,'year_BeltranLeyva_BL0010','Beltran_Leyva',yearL=True,cartelL=True)
Edges_Generetor(dta_0010,'state_year_BL0010','Beltran_Leyva',stateL=True, yearXstate=True)
df_1 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_BL0010.txt')
df_2 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/year_BeltranLeyva_BL0010.txt')
df_BL0010 = pd.concat([df_1,df_2],ignore_index=True, sort=False)

In [45]:
Edges_Generetor(dta_0010,'year_BeltranLeyva_BLF0010','Beltran_Leyva_Family',yearL=True,cartelL=True)
Edges_Generetor(dta_0010,'state_year_BLF0010','Beltran_Leyva_Family',stateL=True, yearXstate=True)
df_1 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/state_year_BLF0010.txt')
df_2 = pd.read_csv('/Users/emanuele/Desktop/SNA/PROGETTO/year_BeltranLeyva_BLF0010.txt')
df_BLF0010 = pd.concat([df_1,df_2],ignore_index=True, sort=False)

In [56]:
x = 'roma'
y = 'r'
x.replace(y,'')

'oma'

In [64]:
df_Z9000 = Edges_Generetor_for_Cartel('Zetas','Z')

In [65]:
df_Z9000

,A,B,Weight
0,2,1992,1
1,2,1993,1
2,2,1994,1
3,2,1999,1
4,7,1998,2
5,9,1994,1
6,11,1994,4
7,15,1994,1
8,15,1996,1
9,15,1999,1
